In [50]:
#import NumPy and Random libraries
import numpy as np
import random


#Randomly generate a 4-bit binary message
def message():
    msg = []
    #randomly select either a zero or one until there are 4 values appended to the msg list
    for i in range(4):
        letter = random.choice([0,1])
        msg.append(letter)
    return np.array(msg)



#Hamming's Generator Matrix (7x4)
G =  np.array([[1, 1, 0, 1],[1, 0, 1, 1],[1, 0, 0, 0],[0, 1, 1, 1], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

#Hamming's Parity-check Matrix (3x7)
H = np.array([[1, 0, 1, 0, 1, 0, 1],[0, 1, 1, 0, 0, 1, 1],[0, 0, 0, 1, 1, 1, 1]])

#Hamming's Decoder Matrix (4x7)
R = np.array([[0,0,1,0,0,0,0], [0,0,0,0,1,0,0], [0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])



#Encode the message using Generator Matrix
#done by multiplying G and the message to get the dot products, this returns the 7-bit codeword
def encode(msg):
    enc = np.dot(G, msg)%2  #modulo of 2, to keep code binary
    return enc

#Check for errors in message after it has been transmitted (gone through the noisy channel) using Parity Check Matrix
#done by multiplying H and the codeword to get the dot products, and returns a 3-bit syndrome vector
def paritycheck(noisy):
    par = np.dot(H, noisy)%2
    return par

#Decode the corrected (if necessary to correct) message back to original 4-bits using Decoder Matrix
def decode(corrected):
    dec = np.dot(R, corrected)%2
    return dec



#Simulate noisy channel by flipping number of bits requested when running function
#bits = random.choice([0,1,2]) would be an alternative route to automate process, 
#but would not allow testing of errors outside of scope of Hamming code
def noise(enc, bits):
    noisy = np.copy(enc)
    #unpack the noisy array to get a list of all indexes (which will serve as the population for the sample)
    index=[*range(0,len(noisy),1)]
    #use random.sample to randomly select the number of indexes for bits requested to be flipped
    e = random.sample(index, bits)

    #iterate through sample and flip the bits for the indexes selected
    for i in e:
        if noisy[i] == 0:
            noisy[i] = 1
        else:
            noisy[i] = 0

    return noisy

#generate error vector
def find_error(par):
    #transpose H so as to group values in each column together
    #creates a "library" of 3-bit arrays to compare the syndrome vector to
    H_trans= H.transpose()
    error_vector=[]
    #iterate through arrays in the matrix and compare them to the syndrome vector
    #create a list by appending 0's where the arrays do not match, and a 1 where they do. 
    for i in H_trans:
        if np.array_equal(i,par):
            error_vector.append(1)
        else:
            error_vector.append(0)

    error_vector=np.array(error_vector)
    return error_vector

#add error vector to noisy vector in order to make the correction 
#by "flipping" the one bit that had previously been flipped back to it's original state
def correct(noisy, error_vector):
    corrected=np.add(noisy,error_vector)%2
    return corrected


#All above functions combined for complete Hamming function w/ clean output
def Hamming(bits):

    msg = message()
    print("4-bit message: ", msg)
    enc = encode(msg)
    print ("Encoded message: ", enc)
    noisy = noise(enc, bits)
    print ('Noisy message: ', noisy)
    par = paritycheck(noisy)
    print('Syndrome vector: ', par)
    error_vec = find_error(par)
    print('Error vector: ', error_vec)
    corrected = correct(noisy, error_vec)
    dec = decode(corrected)
    print ('Decoded message: ', dec)
    
    if np.array_equal(msg,dec) and np.array_equal(par,[0,0,0])==False:
        return "One error has been detented and has been corrected."
    
    elif np.array_equal(msg,dec)==False and np.array_equal(par,[0,0,0])==False:
        return "An error has been detected, but could not be corrected. Please remember, Hamming code is only able to correct one bit errors."
    
    else:
        return "No error detected, but please remember an error may still be present. Hamming code is only able to detect one and two bit errors."
        
        
#Test by changing the input number to how many errors you want introduced:

Hamming(1)

4-bit message:  [1 1 1 1]
Encoded message:  [1 1 1 1 1 1 1]
Noisy message:  [1 1 1 1 1 0 1]
Syndrome vector:  [0 1 1]
Error vector:  [0 0 0 0 0 1 0]
Decoded message:  [1 1 1 1]


'One error has been detented and has been corrected.'